# (10) Autoregressive Model with Elastic Net Estimation (AR_Elastic_Net)

A autoregressive (AR) model with $p$ lags is defined by 

$$
Y_{t} = c + \sum_{i=1}^{p} a_{i}Y_{t-i} + e_{t}.
$$

where $Y_{t}$ represents a growth rates series to be forecasted and $a_{i}$ indicates the coefficients to be determined through model estimation. 

Elastic Net estimation is used to minimize forecast errors. Elastic Net estimation works by adding a penalty term designed to minimize the sum of squared coefficients and the sum of absolute coefficients. Therefore, the coefficients of less important predictors are pushed to zero. In other words, elastic net estimation performs variables selection. Additionally, elastic net takes linear combinations of correlated predictors and works well in cases of multicolinearity.  

$$
L(a_{1},...,a_{n_{a}}) = \sum_{t}(Y_{t+1} - Y_{t+1|t})^{2} + \lambda_{1}\sum_{j=1}^{n_{a}}|a_{j}| + \lambda_{2}\sum_{j=1}^{n_{a}}a_{j}^{2}
$$

The optimal lag length of $p$ is set to a length long enough to return white noise residuals. Reasonable penalty parameters ($\lambda_{1},\lambda_{2}$) are set using validation set root mean squared error (RMSE) minimization. The following code reestimates the AR model each period using walk foreword cross-validation with a fixed lag length over the validation set. Model validation is carried out using an 80-20 split. The initial training model is estimated on the first 80% of the training data. The training model weights are updated after each peiord. Therefore, model weights are always updated to reflect the most recent information. Walk foreword cross-validation is carried out on the remaining 20% of the in-sample set. Each $h$-step ahead forecast is produced using linear model iteration. In the codes below, the phrase "test" actually references the “validation” set AND NOT an out-of-sample test set. 

In the Python Scikit-Learn library, the elastic net loss function is redefined to the following:

$$
L(a_{1},...,a_{n_{a}}) = \sum_{t}(y_{t+1} - f_{t+1|t})^{2} + \alpha \lambda_{1}^{Ratio} \sum_{j=1}^{n_{a}}|a_{j}| + \alpha (1-\lambda_{1}^{Ratio})\sum_{j=1}^{n_{a}}a_{j}^{2}
$$
where $\alpha = \lambda_{1} + \lambda_{2}$ and $\lambda_{1}^{Ratio} = \lambda_{1}/(\lambda_{1} + \lambda_{2})$. Here, $\alpha$ is a homogenous hyperparameter that controls the strength of the penalty. Homogeneity implies that a doubling of $\alpha$ imposes a doubling of each pentalty parameter, both equally and respectively. The Elastic Net Mixture is controlled by the hyperparameter $\lambda_{1}^{Ratio}$. If $\lambda_{1}^{Ratio} = 0$, then the Elastic Net loss function equals the Ridge Regression loss function. If $\lambda_{1}^{Ratio} = 1$, then the Elastic Net loss function equals the Lasso Regression loss function. Therefore, our constraints are $\alpha > 0$ and $0 < \lambda_{1}^{Ratio} < 1$.

The first block of code defines two functions. The DataSpace function takes in the univariate series to be forecasted (target) with the number of lags to use during estimation (lags) and returns the dependent variable to be forecasted (target_series), the dataframe containing the correct number of lags (feature_space), the number of observations in the training set (train_size), and the number of observations in the test set (test_size). The MODEL function takes in six arguments. The dependent series to be forecasted is defined using the target_series argument. The dataframe containing the correct number of lags is defined using the feature_space argument. The number of observations in the training set using the train_size argument. The regularization parameters are set using the penalty and mixture arguments. Lastly, the number of forecast horizons is defined by step_size. The output of the MODEL function is designed to return the training and validation set RMSE values during regularization parameter grid searching. After a reasonable regularization parameter is set into the model, the MODEL function will then return the training and validation set predicted values. The first block of code defines a region to grid search in order to identify the reasonable regularization parameters. The second block of code sets the reasonable regularization parameters into the model and returns the forecasts.   

In [ ]:
# Load Library:
from pandas import read_csv
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
# Function to Create Target Space & Feature Space:
def DataSpace(target, p = 36):
    # Inital Training & Test Set Sizes:
    train_size = int(len(target)*0.8 - p)
    test_size = len(target) - train_size - p
    # Compute Lagged DataFrame:
    target_name = target.columns[0]
    names = [target_name]
    for i in range(0,p,1):
        names.append(target_name+'_L'+str(i+1))
        target[target_name+'_L'+str(i+1)] = target[target_name].shift(i+1)
    target = target.dropna()
    # Split Target Series & Feature Space:
    target_series = target.iloc[:,0]
    feature_space = target.iloc[:, 1:]
    return target_series, feature_space, train_size, test_size
# Function to Fit Model using Walk Foreward Cross-Validation:
def MODEL(target_series, feature_space, train_size, test_size, penalty = 1.0, mixture = 0.5, step_size = 1):
    # Solving the Hyperparameters:
    lambda_1 = penalty*mixture
    lambda_2 = penalty*(1-mixture)
    # Extracting Data:
    index_values = target_series.index.values
    # Storage & Model Estimation:
    test_pred = []
    name = 'Univariate Elastic Net Regression'
    print('-'*len(name))
    print(name)
    print('-'*len(name))
    print('Alpha (Penalty Strength): ', penalty)
    print('L1 Ratio (Mixture): ', mixture)
    for t in range(test_size - step_size + 1):
        # Tracking Convergence:
        print('Test Set Walk Foreward: Iteration '+str(t+1))
        # Define Walk Foreward Training Set:
        target_train = target_series.values[:train_size+t]
        feature_space_train = feature_space.values[:train_size+t, :]
        # Define Walk Foreward Test Set:
        target_test = target_series.values[train_size+t:]
        feature_space_test = feature_space.values[train_size+t:, :]
        # Fit Model to Training Set:
        model = ElasticNet(alpha = penalty, l1_ratio = mixture, random_state = 1)
        model.fit(X = feature_space_train, y = target_train)
        # Evaluate Training Set Performance:
        train_yhat = model.predict(X = feature_space_train)
        # Forecast Storage:
        forecast_storage = feature_space.values[train_size+t,:]
        horizon_storage = []
        for h in range(step_size):
            horizon_storage = np.append(horizon_storage, model.predict(X = forecast_storage[0:feature_space.shape[1]].reshape(1,feature_space.shape[1])))
            forecast_storage = np.insert(forecast_storage, 0, horizon_storage[h])
        # Store Forecasted Values:
        test_pred = np.append(test_pred, horizon_storage[step_size-1])
        # Store Training Predictions:
        if t == 0:
            train_pred = train_yhat
            train_RMSE = np.sqrt(mean_squared_error(target_train, train_yhat))
    # Model Evaluation:
    test_RMSE = np.sqrt(mean_squared_error(target_series.values[train_size + step_size - 1:], test_pred))
    return train_RMSE, test_RMSE, lambda_1, lambda_2 
# Setting Seed:
np.random.seed(12345)
# Load Data:
All_Data = read_csv('Compiled_Data.csv', header = 0, index_col = 0, parse_dates = True)
All_Data.index = pd.DatetimeIndex(All_Data.index.values, freq = "MS")
housing_price = All_Data[['RHP']]
# Create Feature Space & Seperate Target Series:
AR_Lags = 36
target_series, feature_space, train_size, test_size = DataSpace(housing_price, p = AR_Lags)
# Storage for Results & Hyperparameters:
Results = pd.DataFrame(columns = ['Number of Lags', 'Alpha', 'L1 Ratio', 'Lambda 1', 'Lambda 2', 'Train_RMSE', 'Test_RMSE'])
# Setting Hyperparameters:
mixture = np.arange(0.010,0.030,0.001)
penalty = np.arange(0.540,0.560,0.001)
for m in mixture:
    for p in penalty:
        try:
            train_RMSE, test_RMSE, lambda_1, lambda_2 = MODEL(target_series, feature_space, train_size, test_size, penalty = p, mixture = m)
            model_performance = {'Number of Lags':AR_Lags, 'Alpha':p, 'L1 Ratio':m, 'Lambda 1':lambda_1, 'Lambda 2':lambda_2, 'Train_RMSE':train_RMSE, 'Test_RMSE':test_RMSE}
            Results = Results.append(model_performance, ignore_index = True)
        except:
            continue

The second block of code reestimates the top performing model after determining the reasonable regularization parameters.

In [ ]:
# Load Library:
from pandas import read_csv
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
# Function to Create Target Space & Feature Space:
def DataSpace(target, p = 36):
    # Inital Training & Test Set Sizes:
    train_size = int(len(target)*0.8 - p)
    test_size = len(target) - train_size - p
    # Compute Lagged DataFrame:
    target_name = target.columns[0]
    names = [target_name]
    for i in range(0,p,1):
        names.append(target_name+'_L'+str(i+1))
        target[target_name+'_L'+str(i+1)] = target[target_name].shift(i+1)
    target = target.dropna()
    # Split Target Series & Feature Space:
    target_series = target.iloc[:,0]
    feature_space = target.iloc[:, 1:]
    return target_series, feature_space, train_size, test_size
# Function to Fit Model using Walk Foreward Cross-Validation:
def MODEL(target_series, feature_space, train_size, test_size, penalty = 1.0, mixture = 0.5, step_size = 1):
    # Solving the Hyperparameters:
    lambda_1 = penalty*mixture
    lambda_2 = penalty*(1-mixture)
    # Extracting Data:
    index_values = target_series.index.values
    # Storage & Model Estimation:
    test_pred = []
    name = 'Univariate Elastic Net Regression'
    print('-'*len(name))
    print(name)
    print('-'*len(name))
    print('Alpha (Penalty Strength): ', penalty)
    print('L1 Ratio (Mixture): ', mixture)
    for t in range(test_size - step_size + 1):
        # Tracking Convergence:
        print('Test Set Walk Foreward: Iteration '+str(t+1))
        # Define Walk Foreward Training Set:
        target_train = target_series.values[:train_size+t]
        feature_space_train = feature_space.values[:train_size+t, :]
        # Define Walk Foreward Test Set:
        target_test = target_series.values[train_size+t:]
        feature_space_test = feature_space.values[train_size+t:, :]
        # Fit Model to Training Set:
        model = ElasticNet(alpha = penalty, l1_ratio = mixture, random_state = 1)
        model.fit(X = feature_space_train, y = target_train)
        # Evaluate Training Set Performance:
        train_yhat = model.predict(X = feature_space_train)
        # Forecast Storage:
        forecast_storage = feature_space.values[train_size+t,:]
        horizon_storage = []
        for h in range(step_size):
            horizon_storage = np.append(horizon_storage, model.predict(X = forecast_storage[0:feature_space.shape[1]].reshape(1,feature_space.shape[1])))
            forecast_storage = np.insert(forecast_storage, 0, horizon_storage[h])
        # Store Forecasted Values:
        test_pred = np.append(test_pred, horizon_storage[step_size-1])
        # Store Training Predictions:
        if t == 0:
            train_pred = train_yhat
            train_RMSE = np.sqrt(mean_squared_error(target_train, train_yhat))
    # Model Evaluation:
    test_RMSE = np.sqrt(mean_squared_error(target_series.values[train_size + step_size - 1:], test_pred))
    train_pred = pd.DataFrame(train_pred, index = index_values[:train_size], columns = ['train_pred'])
    test_pred = pd.DataFrame(test_pred, index = index_values[train_size + step_size - 1:], columns = ['test_pred'])
    return train_RMSE, test_RMSE, train_pred, test_pred, lambda_1, lambda_2 
# Setting Seed:
np.random.seed(12345)
# Load Data:
All_Data = read_csv('Compiled_Data.csv', header = 0, index_col = 0, parse_dates = True)
All_Data.index = pd.DatetimeIndex(All_Data.index.values, freq = "MS")
housing_price = All_Data[['RHP']]
# Create Feature Space & Seperate Target Series:
AR_Lags = Results.sort_values(by = 'Test_RMSE', ascending = True).iloc[0,0]
target_series, feature_space, train_size, test_size = DataSpace(housing_price, p = AR_Lags)
# Storage for Results & Hyperparameters:
penalty = Results.sort_values(by = 'Test_RMSE', ascending = True).iloc[0,1]
mixture = Results.sort_values(by = 'Test_RMSE', ascending = True).iloc[0,2]
horizons = 1
# Evaluate Model:
train_RMSE, test_RMSE, train_pred, test_pred, lambda_1, lambda_2 = MODEL(target_series, feature_space, train_size, test_size, penalty = penalty, mixture = mixture, step_size = horizons)

The third block presents and graphs the stored output from the MODEL function. The MODEL above is fit to housing price data in order to forecast real housing price growth rates at the U.S. national level.

In [ ]:
# Evaluate Model: Growth Rates
print('-----------------------------')
print('National Housing Price Series')
print('-----------------------------')
print('Data Type: Growth Rates')
print('Model Type: Univariate Elastic Net Regression')
print('Alpha (Penalty Strength) Hyperparameter: ', penalty)
print('L1 Ratio (Mixture) Hyperparameter: ', mixture)
print('Lambda 1 (L1 Hyperparameter): ', lambda_1)
print('Lambda 2 (L2 Hyperparameter): ', lambda_2)
print('Train RMSE: %.3f' % (train_RMSE))
print('Test RMSE: %.3f' % (test_RMSE))
# Plot Forecast: Growth Rates
sns.set_theme(style = 'whitegrid')
pyplot.figure(figsize = (12,6))
pyplot.plot(target_series, label = 'Observed')
pyplot.plot(train_pred, label = 'AR_Elastic_Net: Train')
pyplot.plot(test_pred, label = 'AR_Elastic_Net: Test')
pyplot.xlabel('Date')
pyplot.ylabel('Growth Rate')
pyplot.title('Real Housing Price Series (National)')
pyplot.legend()
pyplot.show()

The fourth block of code is used to analyze the forecast errors for stationarity. The forecast errors are computed, plotted, and distributed. Lastly, the autocorrelation function (ACF) is plotted and the Augmented Dickey-Fuller (ADF) unit root test is carried out.

In [ ]:
# Load Library:
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
# Define Residuals:
resids = target_series[:train_size].values.reshape(train_size,1) - train_pred.values.reshape(train_size,1)
resids = pd.DataFrame(resids, index = target_series.index.values[:train_size], columns = ['resids'])
# Plot Residuals:
sns.set_theme(style = 'whitegrid')
pyplot.figure(figsize = (16,4))
pyplot.subplot(1,2,1)
pyplot.plot(resids)
pyplot.xlabel('Date')
pyplot.title('Residual Series')
pyplot.subplot(1,2,2)
pyplot.hist(resids, bins = 20)
pyplot.title('Residual Distribution')
pyplot.tight_layout()
pyplot.show()
# Plot Autocorelation Function (ACF):
sns.set_theme(style = 'whitegrid')
fig, ax = pyplot.subplots(figsize=(8,4))
plot_acf(resids, title = 'Residual ACF', lags = 36, ax = ax)
pyplot.show()
# ADF Test: Non-Stationary v. Stationary
ADF_Test = adfuller(resids)
print('----------------------')
print('  ADF Unit-Root Test  ')
print('----------------------')
print('Test Statistic: %.3f' % (ADF_Test[0]))
print('P-Value: %.3f' % (ADF_Test[1]))
print('Critical Values:')
for key, value in ADF_Test[4].items():
    print('%s: %.3f' % (key, value))

The last block of code loads in the previous .csv files "National_Train_Growth_One" and "National_Test_Growth_One" that contain the stored forecasted values. The storage files are then augmented to include the predicted values from the current algorithm in order to estimate the forecast combinations, produce the final "top performing" model plots, and carry out the final comparison tests for predictive accuracy.

In [ ]:
# Load Forecast Tables: 
train_forecasts = read_csv('National_Train_Growth_One.csv', header = 0, index_col = 0, parse_dates = True)
train_forecasts.index = pd.DatetimeIndex(train_forecasts.index.values, freq = "MS")
test_forecasts = read_csv('National_Test_Growth_One.csv', header = 0, index_col = 0, parse_dates = True)
test_forecasts.index = pd.DatetimeIndex(test_forecasts.index.values, freq = "MS")
# Add New Forecast Model:
train_forecasts['AR_Elastic_Net'] = train_pred
test_forecasts['AR_Elastic_Net'] = test_pred
# Save Forecast:
pd.DataFrame(train_forecasts).to_csv('National_Train_Growth_One.csv')
pd.DataFrame(test_forecasts).to_csv('National_Test_Growth_One.csv')